In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm
from object_function import * 
from shrinkage_method import *

In [2]:
price = pd.read_pickle("C:\\Jehyeon\\Portfolio-Optimization-2023-FR-Project-\\Data\\allstock_reduced.pickle")
spx_mask = pd.read_pickle("C:\\Jehyeon\\Portfolio-Optimization-2023-FR-Project-\\Data\\spx_mask.pickle")

In [3]:
rtn = price.pct_change(fill_method=None)

In [4]:
pd.date_range(start="2020",
              end="2022",freq="QS")

DatetimeIndex(['2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01',
               '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01',
               '2022-01-01'],
              dtype='datetime64[ns]', freq='QS-JAN')

In [5]:
pd.date_range(start="2020",
              end="2022",freq="Q")

DatetimeIndex(['2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31',
               '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31'],
              dtype='datetime64[ns]', freq='Q-DEC')

In [6]:
def run_optimizer(rtn_df:pd.DataFrame, spx_mask:pd.DataFrame,
                  start_year:str, end_year:str, 
                  rebalancing:str, shrinkage_method="None"):
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    shrink = {"None": no_shrinkage,
              "linear":linear_shrinkage,
              "constant":constant_correlation_model,
              "clipping":eigenvalue_clipping,
              "clustering":kmeans_clustering}

    weight_df = pd.DataFrame(columns=rtn_df.columns) # weight를 담을 dataframe

    start_idx = pd.date_range(start_year,end_year, freq=f"{rebalancing}S")
    end_idx = pd.date_range(start_year,end_year, freq=f"{rebalancing}")

    for rebalancing_date in tqdm(end_idx):
            # start~end의 주가를 보고 포폴 구성(Look Back Window는 1년이 된다)
            start = (rebalancing_date - pd.Timedelta(days=365)).strftime("%Y-%m") 
            end = (rebalancing_date - pd.Timedelta(days=1))             
            
            mask_sample = spx_mask.loc[:end].iloc[-1]
            universe = mask_sample.loc[~mask_sample.isna()].index # S&P500 구성종목을 가져옵니다
            rtn_lookback = rtn_df.loc[start:end, universe] 
           
            rtn_vol = np.diag(rtn_lookback.std())
            corr_matrix = rtn_lookback.corr() # corr_matrix를 추정하고, optimizer에 넣기 전에 cov_matrix로 변환해야함
            
            mean_return = rtn_lookback.mean()
            
            if shrinkage_method == "None" or shrinkage_method == "constant":
                args = {"args":0}
            
            shrinked_corr_matrix = shrink[shrinkage_method](corr_matrix = corr_matrix, **args)
            cov_matrix = rtn_vol.dot(shrinked_corr_matrix).dot(rtn_vol) # corr matrix를 cov matrix로 변경
            
            if shrinkage_method == "clustering":
                self.k_dict[start_idx[i]] = self.k
                        
            bounds = tuple((0,0.05) for _ in range(len(rtn_lookback.columns)))
            initial_weights = np.ones(len(rtn_lookback.columns)) / len(rtn_lookback.columns)
            
            # 최적화 수행
            result = minimize(self.__obj_sharpe, 
                              initial_weights, 
                              args=(cov_matrix,),
                              method='SLSQP', 
                              constraints=constraints, 
                              bounds=bounds
                              )
            min_variance_weights = result.x
            weight_df.loc[start_idx[i], universe] = min_variance_weights
        
        print("Jobs Done...")
        print("You can check .rebalancing_date")
        return weight_df